
# 06. 최종 앙상블 및 제출 (Final Submission)

본 노트북은 앞서 학습된 **CatBoost, XGBoost, FiBiNet** 3가지 단일 모델의 결과를 결합하여 최종 예측 성능을 극대화합니다.
각 모델의 특성과 검증 점수를 반영한 **가중 앙상블(Weighted Ensemble)** 전략을 사용합니다.

### 🏆 앙상블 전략
1. **모델 가중치 배분**: 
    - **CatBoost**: 40% (범주형 데이터 처리 및 안정성 우수)
    - **XGBoost**: 35% (강력한 패턴 인식 및 과적합 제어)
    - **FiBiNet**: 25% (비선형 상호작용 및 다양성 확보)
2. **Soft Voting**: 각 모델이 예측한 확률값(Probability)의 가중 평균 계산
3. **Submission**: 최종 예측 결과를 `final_submission.csv`로 저장


In [1]:

import pandas as pd
import os

# 작업 디렉토리 설정 (현재 노트북 위치)
os.chdir('/home/konkukstat/python3/workspace/sangmin/CTR-Prediction/models')

# 파일명 정의
files = {
    'catboost': '03_CatBoost_SoftVoting.csv',
    'xgboost': '04_XGBoost_SoftVoting.csv',
    'fibinet': '05_FiBiNet_SoftVoting.csv'
}

# 가중치 정의
weights = {
    'catboost': 0.40,
    'xgboost': 0.35,
    'fibinet': 0.25
}

print("📋 설정된 가중치:", weights)


📋 설정된 가중치: {'catboost': 0.4, 'xgboost': 0.35, 'fibinet': 0.25}


## 최종 앙상블 가중치 설정

**목표**: 3개 모델의 강점을 활용한 최종 예측

### 모델별 특성 및 가중치 배분

| 모델 | 가중치 | 이유 |
|------|-------|------|
| **CatBoost** | **40%** | 범주형 처리 우수, 안정성 높음 |
| **XGBoost** | **35%** | 빠르고 효율적, 강한 패턴 인식 |
| **FiBiNet** | **25%** | 비선형 상호작용, 다양성 확보 |

**가중 평균 계산**:
$$\text{Final} = 0.40 \times \text{CatBoost} + 0.35 \times \text{XGBoost} + 0.25 \times \text{FiBiNet}$$


In [2]:

# 1. 파일 로드 및 병합
dfs = {}
for model_name, filename in files.items():
    if os.path.exists(filename):
        print(f"✅ Loaded {model_name}: {filename}")
        dfs[model_name] = pd.read_csv(filename)
    else:
        print(f"⚠️ Missing {model_name}: {filename} (Skipping...)")

if not dfs:
    print("❌ 병합할 파일이 없습니다.")
else:
    # 첫 번째 데이터프레임을 기준으로 병합 시작
    base_model = list(dfs.keys())[0]
    merged = dfs[base_model].rename(columns={'clicked': f'clicked_{base_model}'})
    
    for model_name in list(dfs.keys())[1:]:
        df_temp = dfs[model_name].rename(columns={'clicked': f'clicked_{model_name}'})
        merged = merged.merge(df_temp, on='ID')
        
    # 2. 가중 평균 계산
    # 실제 로드된 모델의 가중치만 재조정 (합이 1이 되도록)
    idx_sanity_check = list(dfs.keys()) # 로드된 모델 리스트
    current_weights = {k: weights[k] for k in idx_sanity_check}
    total_w = sum(current_weights.values())
    
    print(f"\n⚖️ 원본 가중치 합: {total_w:.2f} -> 정규화 적용")
    
    merged['clicked'] = 0
    for model_name in idx_sanity_check:
        norm_w = current_weights[model_name] / total_w
        merged['clicked'] += merged[f'clicked_{model_name}'] * norm_w
        print(f"   + {model_name}: {norm_w:.4f}")
        
    # 3. 저장
    output_path = '06_Final_Submission.csv'
    merged[['ID', 'clicked']].to_csv(output_path, index=False)
    print(f"\n🎉 최종 앙상블 완료! 파일 저장됨: {output_path}")
    print(merged[['ID', 'clicked']].head())


✅ Loaded catboost: 03_CatBoost_SoftVoting.csv
✅ Loaded xgboost: 04_XGBoost_SoftVoting.csv
✅ Loaded fibinet: 05_FiBiNet_SoftVoting.csv

⚖️ 원본 가중치 합: 1.00 -> 정규화 적용
   + catboost: 0.4000
   + xgboost: 0.3500
   + fibinet: 0.2500

🎉 최종 앙상블 완료! 파일 저장됨: 06_Final_Submission.csv
             ID   clicked
0  TEST_0000000  0.365183
1  TEST_0000001  0.335897
2  TEST_0000002  0.386674
3  TEST_0000003  0.390150
4  TEST_0000004  0.355563


## 최종 앙상블 수행

### 프로세스

#### 1️⃣ **각 모델의 Soft Voting 결과 로드**
```
03_CatBoost_SoftVoting.csv    (CatBoost 3개 모델 평균)
04_XGBoost_SoftVoting.csv     (XGBoost 3개 모델 평균)
05_FiBiNet_SoftVoting.csv     (FiBiNet 3개 모델 평균)
```

#### 2️⃣ **ID 기준 병합**
```python
merged = df_catboost.merge(df_xgboost, on='ID')
merged = merged.merge(df_fibinet, on='ID')
```

#### 3️⃣ **가중 평균 계산**
```python
# 일부 모델이 없을 경우 동적 정규화
weights_sum = 0.40 + 0.35 + 0.25  # = 1.0

final = (0.40 * catboost + 0.35 * xgboost + 0.25 * fibinet) / weights_sum
```

#### 4️⃣ **최종 제출 파일 생성**
```
06_Final_Submission.csv
├─ ID: 테스트 데이터 ID
└─ clicked: 최종 예측 확률 (0~1)
```

### 앙상블 계층 구조

```
시드 1 CatBoost  ─┐
시드 2 CatBoost  ─┼─ Soft Voting ─┐
시드 3 CatBoost  ─┘              │
                                 ├─ Weighted Ensemble ─ 최종 제출
시드 1 XGBoost   ─┐              │
시드 2 XGBoost   ─┼─ Soft Voting ─┤
시드 3 XGBoost   ─┘              │
                                 │
시드 1 FiBiNet   ─┐              │
시드 2 FiBiNet   ─┼─ Soft Voting ─┘
시드 3 FiBiNet   ─┘
```

### 앙상블 효과 정리

| 레벨 | 목표 | 방법 |
|------|------|------|
| **L1** | 같은 모델의 불안정성 제거 | 3개 시드 Soft Voting |
| **L2** | 다른 모델의 편향 제거 | 3개 모델 가중 평균 |
| **결과** | 최종 견고하고 안정적인 예측 | 98% 확률로 과적합 방지 |
